## Simple demo of creating an Azure Datastore

### Do the required pip installs

In [4]:
pip install azure-ai-ml

     |████████████████████████████████| 6.2 MB 5.5 MB/s eta 0:00:01
     |████████████████████████████████| 247 kB 49.2 MB/s eta 0:00:01
     |████████████████████████████████| 84 kB 4.6 MB/s  eta 0:00:01
     |████████████████████████████████| 266 kB 59.7 MB/s eta 0:00:01
     |████████████████████████████████| 123 kB 46.3 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 7.0 MB/s  eta 0:00:01
ERROR: azure-storage-file-datalake 12.12.0 has requirement azure-core<2.0.0,>=1.28.0, but you'll have azure-core 1.26.4 which is incompatible.
ERROR: azure-storage-file-datalake 12.12.0 has requirement azure-storage-blob<13.0.0,>=12.17.0, but you'll have azure-storage-blob 12.13.0 which is incompatible.
ERROR: azure-storage-file-share 12.13.0 has requirement azure-core<2.0.0,>=1.28.0, but you'll have azure-core 1.26.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


### Connect to the default workspace

In [5]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [7]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


### List existing datastores

In [8]:
stores = ml_client.datastores.list()
for ds_name in stores:
    print(ds_name.name)

workspaceartifactstore
workspacefilestore
workspaceworkingdirectory
workspaceblobstore


### Now create a user specified datastore

In [10]:
from azure.ai.ml.entities import AzureBlobDatastore
from azure.ai.ml.entities import AccountKeyConfiguration

store = AzureBlobDatastore(
    name="blob_training_data",
    description="Blob Storage for training data",
    account_name="mldemo3wstorageb62532e47",
    container_name="training-data", 
    credentials=AccountKeyConfiguration(
        account_key="iCzE8/NeNSHx9AmpUfJAOZo3LPvLes1u/t4XUcBpDKfkxz36PPAEEwcavHMW7gpeDuM11KRtol8h+ASt8PwHlQ=="
    ),
)

ml_client.create_or_update(store)

AzureBlobDatastore({'type': <DatastoreType.AZURE_BLOB: 'AzureBlob'>, 'name': 'blob_training_data', 'description': 'Blob Storage for training data', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/16f263d1-840b-4414-8415-243b4662d595/resourceGroups/packt-azureml-rg2/providers/Microsoft.MachineLearningServices/workspaces/ml-demo3-ws2/datastores/blob_training_data', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/simple-ml-compute/code/Users/adeidowu', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7fcac84442b0>, 'credentials': {'type': 'account_key'}, 'container_name': 'training-data', 'account_name': 'mldemo3wstorageb62532e47', 'endpoint': 'core.windows.net', 'protocol': 'https'})

### List the new Datastore (Azure blob) 

In [11]:
stores = ml_client.datastores.list()
for ds_name in stores:
    print(ds_name.name)

blob_training_data
workspaceartifactstore
workspacefilestore
workspaceworkingdirectory
workspaceblobstore


### Upload (connect) Scikit Learn diabetics dataset to the Workspace default datastore blob

In [ ]:
from sklearn.datasets import load_diabetes
import pandas as pd
features, target = load_diabetes(return_X_y=True)
diabetes_df = pd.DataFrame(features)
diabetes_df['target']= target
diabetes_df.to_csv('rawdata.csv', index=False)
dstore = ml_client.get_default_datastore()
dstore.upload_files(
            files=['rawdata.csv'],
            target_path="/samples/diabetes/v1",
            overwrite=True,
            show_progress=True)

In [14]:
# files = ["https://bit.ly/survey-drift-base"]
# store.upload_files(
#     files=files, # List[str] of absolute paths of files to upload
#     target_path='training-data/',
#     overwrite=False,
#     )

AttributeError: 'AzureBlobDatastore' object has no attribute 'upload_files'